In [7]:
import pandas as pd
import numpy as np
from untrade.client import Client

In [8]:
import pandas as pd
import numpy as np

def process_data(df):
    """
    Process the input dataframe to calculate indicators, generate trade signals,
    and apply stop-loss (SL) and take-profit (TP) logic based on the new strategy.

    Parameters:
    - df (DataFrame): Input dataframe with OHLC and volume data.

    Returns:
    - df (DataFrame): Processed dataframe with signals, trade types, and additional columns.
    """
    df = df.copy()
    
    # Calculate moving averages
    df['MA_13'] = df['close'].rolling(window=13).mean()
    df['MA_15'] = df['close'].rolling(window=15).mean()
    df['MA_200'] = df['close'].rolling(window=200).mean()
    
    # Calculate Bollinger Bands (20-period SMA with 2 standard deviations)
    df['BB_Mid'] = df['close'].rolling(window=20).mean()
    df['BB_Upper'] = df['BB_Mid'] + (2 * df['close'].rolling(window=20).std())
    df['BB_Lower'] = df['BB_Mid'] - (2 * df['close'].rolling(window=20).std())
    
    # Calculate average volume over the last 26 periods
    df['Avg_Volume'] = df['volume'].rolling(window=26).mean()
    
    # Initialize trade columns
    df['signals'] = 0  # Buy (1), Sell (-1), Square-off (0)
    df['trade'] = 0  # Track active trades (1 for long, -1 for short)
    df['SL'] = np.nan  # Stop-loss levels
    df['TP'] = np.nan  # Take-profit levels
    df['trade_type'] = 'square-off'  # Trade type ('long', 'short', 'square-off')
    
    # Loop through data to calculate signals and update trade states
    for i in range(200, len(df)):
        curr_close = df['close'].iloc[i]
        curr_low = df['low'].iloc[i]
        curr_high = df['high'].iloc[i]
        curr_vol = df['volume'].iloc[i]
        avg_vol = df['Avg_Volume'].iloc[i]
        prev_trade = df['trade'].iloc[i-1]
        
        # Check if volume condition is met
        vol_condition = curr_vol >= 1.3 * avg_vol
        
        # Long condition
        if df['MA_13'].iloc[i] > df['MA_15'].iloc[i] and curr_close > df['MA_200'].iloc[i] and vol_condition:
            if prev_trade == -1:  # Square-off short trade
                df.loc[df.index[i], 'signals'] = -1
                df.loc[df.index[i], 'trade'] = 0
                df.loc[df.index[i], 'trade_type'] = 'square-off'
            else:  # Open long trade
                delta = curr_close - curr_low
                df.loc[df.index[i], 'signals'] = 1
                df.loc[df.index[i], 'SL'] = curr_low - (0.1 * delta)
                df.loc[df.index[i], 'TP'] = df['BB_Upper'].iloc[i]
                df.loc[df.index[i], 'trade'] = 1
                df.loc[df.index[i], 'trade_type'] = 'long'
        
        # Short condition
        elif df['MA_13'].iloc[i] < df['MA_15'].iloc[i] and curr_close < df['MA_200'].iloc[i] and vol_condition:
            if prev_trade == 1:  # Square-off long trade
                df.loc[df.index[i], 'signals'] = 1
                df.loc[df.index[i], 'trade'] = 0
                df.loc[df.index[i], 'trade_type'] = 'square-off'
            else:  # Open short trade
                delta = curr_high - curr_close
                df.loc[df.index[i], 'signals'] = -1
                df.loc[df.index[i], 'SL'] = curr_high + (0.1 * delta)
                df.loc[df.index[i], 'TP'] = df['BB_Lower'].iloc[i]
                df.loc[df.index[i], 'trade'] = -1
                df.loc[df.index[i], 'trade_type'] = 'short'
        else:
            df.loc[df.index[i], 'trade'] = prev_trade  # Maintain trade state
    
    # Keep only required columns for backtesting
    df = df[['datetime', 'open', 'high', 'low', 'close', 'volume', 'signals', 'trade_type', 'SL', 'TP', 'MA_13', 'MA_15', 'MA_200', 'BB_Upper', 'BB_Lower']]
    
    return df



def strat(data):
    """
    Filter out consecutive duplicate signals to ensure no repetitive trades.

    Parameters:
    - data (DataFrame): Dataframe with trade signals.

    Returns:
    - data (DataFrame): Dataframe with filtered signals.
    """
    signal = []
    prev = None
    for value in data["signals"]:
        if value == prev:
            signal.append(0)  # No action if same signal as previous
        else:
            signal.append(value)  # Keep the signal if different
        prev = value

    data["signals"] = signal

    # Retain only necessary columns
    data = data[['datetime', 'open', 'high', 'low',
                'close', 'volume', 'signals', 'trade_type']]
    
    data["leverage"] = 5
    data["position"] = 100
    data["quantity"] = 1000
    return data


def perform_backtest(csv_file_path):
    """
    Perform backtesting using the Untrade SDK.

    Parameters:
    - csv_file_path (str): Path to the CSV file with processed data.

    Returns:
    - result (generator): Backtest results generator.
    """
    # Create Untrade client instance
    client = Client()

    # Perform backtest using Untrade
    result = client.backtest(
        jupyter_id="team64_zelta_hpps",  # User's Jupyter ID for Untrade
        file_path=csv_file_path,
        leverage=5,  # Set leverage for backtest
        position=50,  # Set initial position sizeq
        quantity=1000,  # Set quantity for backtest
    )

    return result

# Load ETH data


data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/ETH/ETHUSDT_4h.csv")


# Process data
res = process_data(data)

# Apply filtering strategy to remove duplicate signals
res = strat(res)

# Save processed data to CSV for backtesting
res.to_csv("processed_data.csv", index=False)

# Perform backtesting
csv_file_path = "processed_data.csv"
backtest_result = perform_backtest(csv_file_path)

# Print backtest results
for value in backtest_result:
    print(value)


/var/folders/55/_kpfspls6xq0pzzm_dl4ylxc0000gn/T/ipykernel_1512/3996607727.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["leverage"] = 5
/var/folders/55/_kpfspls6xq0pzzm_dl4ylxc0000gn/T/ipykernel_1512/3996607727.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["position"] = 100
/var/folders/55/_kpfspls6xq0pzzm_dl4ylxc0000gn/T/ipykernel_1512/3996607727.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

data: {
  "jupyter_id": "team64-5fzelta-5fhpps",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-12-01 00:00:00",
      "Total Trades": 61,
      "Leverage Applied": 5.0,
      "Winning Trades": 24,
      "Losing Trades": 37,
      "No. of Long Trades": 30,
      "No. of Short Trades": 31,
      "Benchmark Return(%)": 1447.07322,
      "Benchmark Return(on $1000)": 14470.732205,
      "Win Rate": 39.344262,
      "Winning Streak": 6,
      "Losing Streak": 5,
      "Gross Profit": 17791.858056,
      "Net Profit": 17334.358056,
      "Average Profit": 284.169804,
      "Maximum Drawdown(%)": 35.034851,
      "Average Drawdown(%)": 6.265724,
      "Largest Win": 6364.174491,
      "Average Win": 1315.866931,
      "Largest Loss": -1635.451994,
      "Average Loss": -385.039143,
      "Maximum Holding Time": "94 days 0:0:0",
      "Average Holding Time": "21 days 5:14:45",
      "Maximum Adverse Excursion": 48.84106

In [9]:
#ETHUSDT_12h.csv ETHUSDT_15m.csv ETHUSDT_1d.csv  ETHUSDT_1h.csv  ETHUSDT_2h.csv  ETHUSDT_30m.csv ETHUSDT_4h.csv  ETHUSDT_6h.csv  ETHUSDT_8h.csv

data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/ETH/ETHUSDT_4h.csv")


# Process data
res = process_data(data)

# Apply filtering strategy to remove duplicate signals
res = strat(res)

# Save processed data to CSV for backtesting
res.to_csv("processed_data.csv", index=False)

# Perform backtesting
csv_file_path = "processed_data.csv"
backtest_result = perform_backtest(csv_file_path)

# Print backtest results
for value in backtest_result:
    print(value)


/var/folders/55/_kpfspls6xq0pzzm_dl4ylxc0000gn/T/ipykernel_1512/3996607727.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["leverage"] = 5
/var/folders/55/_kpfspls6xq0pzzm_dl4ylxc0000gn/T/ipykernel_1512/3996607727.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["position"] = 100
/var/folders/55/_kpfspls6xq0pzzm_dl4ylxc0000gn/T/ipykernel_1512/3996607727.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

data: {
  "jupyter_id": "team64-5fzelta-5fhpps",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-12-01 00:00:00",
      "Total Trades": 61,
      "Leverage Applied": 5.0,
      "Winning Trades": 24,
      "Losing Trades": 37,
      "No. of Long Trades": 30,
      "No. of Short Trades": 31,
      "Benchmark Return(%)": 1447.07322,
      "Benchmark Return(on $1000)": 14470.732205,
      "Win Rate": 39.344262,
      "Winning Streak": 6,
      "Losing Streak": 5,
      "Gross Profit": 17791.858056,
      "Net Profit": 17334.358056,
      "Average Profit": 284.169804,
      "Maximum Drawdown(%)": 35.034851,
      "Average Drawdown(%)": 6.265724,
      "Largest Win": 6364.174491,
      "Average Win": 1315.866931,
      "Largest Loss": -1635.451994,
      "Average Loss": -385.039143,
      "Maximum Holding Time": "94 days 0:0:0",
      "Average Holding Time": "21 days 5:14:45",
      "Maximum Adverse Excursion": 48.84106

In [10]:
#ETHUSDT_12h.csv ETHUSDT_15m.csv ETHUSDT_1d.csv  ETHUSDT_1h.csv  ETHUSDT_2h.csv  ETHUSDT_30m.csv ETHUSDT_4h.csv  ETHUSDT_6h.csv  ETHUSDT_8h.csv

data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/ETH/ETHUSDT_6h.csv")


# Process data
res = process_data(data)

# Apply filtering strategy to remove duplicate signals
res = strat(res)

# Save processed data to CSV for backtesting
res.to_csv("processed_data.csv", index=False)

# Perform backtesting
csv_file_path = "processed_data.csv"
backtest_result = perform_backtest(csv_file_path)

# Print backtest results
for value in backtest_result:
    print(value)


/var/folders/55/_kpfspls6xq0pzzm_dl4ylxc0000gn/T/ipykernel_1512/3996607727.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["leverage"] = 5
/var/folders/55/_kpfspls6xq0pzzm_dl4ylxc0000gn/T/ipykernel_1512/3996607727.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["position"] = 100
/var/folders/55/_kpfspls6xq0pzzm_dl4ylxc0000gn/T/ipykernel_1512/3996607727.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

data: {
  "jupyter_id": "team64-5fzelta-5fhpps",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-12-01 00:00:00",
      "Total Trades": 32,
      "Leverage Applied": 5.0,
      "Winning Trades": 16,
      "Losing Trades": 16,
      "No. of Long Trades": 16,
      "No. of Short Trades": 16,
      "Benchmark Return(%)": 1442.288422,
      "Benchmark Return(on $1000)": 14422.884225,
      "Win Rate": 50.0,
      "Winning Streak": 3,
      "Losing Streak": 4,
      "Gross Profit": 26603.449861,
      "Net Profit": 26363.449861,
      "Average Profit": 823.857808,
      "Maximum Drawdown(%)": 7.653176,
      "Average Drawdown(%)": 2.054125,
      "Largest Win": 15562.357243,
      "Average Win": 2057.342426,
      "Largest Loss": -1183.844828,
      "Average Loss": -409.62681,
      "Maximum Holding Time": "162 days 12:0:0",
      "Average Holding Time": "40 days 21:11:15",
      "Maximum Adverse Excursion": 60.329594,

In [11]:
#ETHUSDT_12h.csv ETHUSDT_15m.csv ETHUSDT_1d.csv  ETHUSDT_1h.csv  ETHUSDT_2h.csv  ETHUSDT_30m.csv ETHUSDT_4h.csv  ETHUSDT_6h.csv  ETHUSDT_8h.csv

data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/ETH/ETHUSDT_8h.csv")


# Process data
res = process_data(data)

# Apply filtering strategy to remove duplicate signals
res = strat(res)

# Save processed data to CSV for backtesting
res.to_csv("processed_data.csv", index=False)

# Perform backtesting
csv_file_path = "processed_data.csv"
backtest_result = perform_backtest(csv_file_path)

# Print backtest results
for value in backtest_result:
    print(value)


/var/folders/55/_kpfspls6xq0pzzm_dl4ylxc0000gn/T/ipykernel_1512/3996607727.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["leverage"] = 5
/var/folders/55/_kpfspls6xq0pzzm_dl4ylxc0000gn/T/ipykernel_1512/3996607727.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["position"] = 100
/var/folders/55/_kpfspls6xq0pzzm_dl4ylxc0000gn/T/ipykernel_1512/3996607727.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

data: {
  "jupyter_id": "team64-5fzelta-5fhpps",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-12-01 00:00:00",
      "Total Trades": 26,
      "Leverage Applied": 5.0,
      "Winning Trades": 10,
      "Losing Trades": 16,
      "No. of Long Trades": 13,
      "No. of Short Trades": 13,
      "Benchmark Return(%)": 1457.640166,
      "Benchmark Return(on $1000)": 14576.401662,
      "Win Rate": 38.461538,
      "Winning Streak": 3,
      "Losing Streak": 4,
      "Gross Profit": 15203.15889,
      "Net Profit": 15008.15889,
      "Average Profit": 577.23688,
      "Maximum Drawdown(%)": 320.420685,
      "Average Drawdown(%)": 22.387087,
      "Largest Win": 16505.296336,
      "Average Win": 2633.767471,
      "Largest Loss": -2969.122329,
      "Average Loss": -708.094739,
      "Maximum Holding Time": "156 days 0:0:0",
      "Average Holding Time": "48 days 23:41:32",
      "Maximum Adverse Excursion": 65.44

In [12]:
#ETHUSDT_12h.csv ETHUSDT_15m.csv ETHUSDT_1d.csv  ETHUSDT_1h.csv  ETHUSDT_2h.csv  ETHUSDT_30m.csv ETHUSDT_4h.csv  ETHUSDT_6h.csv  ETHUSDT_8h.csv

data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/ETH/ETHUSDT_12h.csv")


# Process data
res = process_data(data)

# Apply filtering strategy to remove duplicate signals
res = strat(res)

# Save processed data to CSV for backtesting
res.to_csv("processed_data.csv", index=False)

# Perform backtesting
csv_file_path = "processed_data.csv"
backtest_result = perform_backtest(csv_file_path)

# Print backtest results
for value in backtest_result:
    print(value)


/var/folders/55/_kpfspls6xq0pzzm_dl4ylxc0000gn/T/ipykernel_1512/3996607727.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["leverage"] = 5
/var/folders/55/_kpfspls6xq0pzzm_dl4ylxc0000gn/T/ipykernel_1512/3996607727.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["position"] = 100
/var/folders/55/_kpfspls6xq0pzzm_dl4ylxc0000gn/T/ipykernel_1512/3996607727.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

In [13]:
import pandas as pd
import numpy as np

def process_data(df):
    """
    Process the input dataframe to calculate indicators, generate trade signals,
    and apply stop-loss (SL) and take-profit (TP) logic based on the new strategy.

    Parameters:
    - df (DataFrame): Input dataframe with OHLC and volume data.

    Returns:
    - df (DataFrame): Processed dataframe with signals, trade types, and additional columns.
    """
    df = df.copy()
    
    # Calculate moving averages
    df['MA_13'] = df['close'].rolling(window=13).mean()
    df['MA_15'] = df['close'].rolling(window=15).mean()
    df['MA_200'] = df['close'].rolling(window=200).mean()
    
    # Calculate Bollinger Bands (20-period SMA with 2 standard deviations)
    df['BB_Mid'] = df['close'].rolling(window=20).mean()
    df['BB_Upper'] = df['BB_Mid'] + (2 * df['close'].rolling(window=20).std())
    df['BB_Lower'] = df['BB_Mid'] - (2 * df['close'].rolling(window=20).std())
    
    # Calculate average volume over the last 26 periods
    df['Avg_Volume'] = df['volume'].rolling(window=26).mean()
    
    # Initialize trade columns
    df['signals'] = 0  # Buy (1), Sell (-1), Square-off (0)
    df['trade'] = 0  # Track active trades (1 for long, -1 for short)
    df['SL'] = np.nan  # Stop-loss levels
    df['TP'] = np.nan  # Take-profit levels
    df['trade_type'] = 'square-off'  # Trade type ('long', 'short', 'square-off')
    
    # Loop through data to calculate signals and update trade states
    for i in range(200, len(df)):
        curr_close = df['close'].iloc[i]
        curr_low = df['low'].iloc[i]
        curr_high = df['high'].iloc[i]
        curr_vol = df['volume'].iloc[i]
        avg_vol = df['Avg_Volume'].iloc[i]
        prev_trade = df['trade'].iloc[i-1]
        
        # Check if volume condition is met
        vol_condition = curr_vol >= 1.3 * avg_vol
        
        # Long condition
        if df['MA_13'].iloc[i] > df['MA_15'].iloc[i] and curr_close > df['MA_200'].iloc[i] and vol_condition:
            if prev_trade == -1:  # Square-off short trade
                df.loc[df.index[i], 'signals'] = -1
                df.loc[df.index[i], 'trade'] = 0
                df.loc[df.index[i], 'trade_type'] = 'square-off'
            else:  # Open long trade
                delta = curr_close - curr_low
                df.loc[df.index[i], 'signals'] = 1
                df.loc[df.index[i], 'SL'] = curr_low - (0.1 * delta)
                df.loc[df.index[i], 'TP'] = df['BB_Upper'].iloc[i]
                df.loc[df.index[i], 'trade'] = 1
                df.loc[df.index[i], 'trade_type'] = 'long'
        
        # Short condition
        elif df['MA_13'].iloc[i] < df['MA_15'].iloc[i] and curr_close < df['MA_200'].iloc[i] and vol_condition:
            if prev_trade == 1:  # Square-off long trade
                df.loc[df.index[i], 'signals'] = 1
                df.loc[df.index[i], 'trade'] = 0
                df.loc[df.index[i], 'trade_type'] = 'square-off'
            else:  # Open short trade
                delta = curr_high - curr_close
                df.loc[df.index[i], 'signals'] = -1
                df.loc[df.index[i], 'SL'] = curr_high + (0.1 * delta)
                df.loc[df.index[i], 'TP'] = df['BB_Lower'].iloc[i]
                df.loc[df.index[i], 'trade'] = -1
                df.loc[df.index[i], 'trade_type'] = 'short'
        else:
            df.loc[df.index[i], 'trade'] = prev_trade  # Maintain trade state
    
    # Keep only required columns for backtesting
    df = df[['datetime', 'open', 'high', 'low', 'close', 'volume', 'signals', 'trade_type', 'SL', 'TP', 'MA_13', 'MA_15', 'MA_200', 'BB_Upper', 'BB_Lower']]
    
    return df



def strat(data):
    """
    Filter out consecutive duplicate signals to ensure no repetitive trades.

    Parameters:
    - data (DataFrame): Dataframe with trade signals.

    Returns:
    - data (DataFrame): Dataframe with filtered signals.
    """
    data = data.copy()
    signal = []
    prev = None
    for value in data["signals"]:
        if value == prev:
            signal.append(0)  # No action if same signal as previous
        else:
            signal.append(value)  # Keep the signal if different
        prev = value

    data["signals"] = signal
    

    # Retain only necessary columns
    data = data[['datetime', 'open', 'high', 'low',
                'close', 'volume', 'signals', 'trade_type']]
    
    data['ATR'] = data['high'].rolling(14).max() - data['low'].rolling(14).min()

    # Normalize ATR for relative volatility
    data['ATR_normalized'] = data['ATR'] / data['close']
    
    data["leverage"] = 5
    data["position"] = np.where(data["ATR_normalized"] < data["ATR_normalized"].quantile(0.5), 100, 50)
    return data


def perform_backtest(csv_file_path):
    """
    Perform backtesting using the Untrade SDK.

    Parameters:
    - csv_file_path (str): Path to the CSV file with processed data.

    Returns:
    - result (generator): Backtest results generator.
    """
    # Create Untrade client instance
    client = Client()

    # Perform backtest using Untrade
    result = client.backtest(
        jupyter_id="team64_zelta_hpps",  # User's Jupyter ID for Untrade
        file_path=csv_file_path,
        leverage=5,  # Set leverage for backtest
        # position=50,  # Set initial position sizeq
    )

    return result

# Load ETH data


data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/ETH/ETHUSDT_4h.csv")


# Process data
res = process_data(data)

# Apply filtering strategy to remove duplicate signals
res = strat(res)

# Save processed data to CSV for backtesting
res.to_csv("processed_data.csv", index=False)

# Perform backtesting
csv_file_path = "processed_data.csv"
backtest_result = perform_backtest(csv_file_path)

# Print backtest results
for value in backtest_result:
    print(value)


data: {
  "jupyter_id": "team64-5fzelta-5fhpps",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-12-01 00:00:00",
      "Total Trades": 61,
      "Leverage Applied": 5.0,
      "Winning Trades": 24,
      "Losing Trades": 37,
      "No. of Long Trades": 30,
      "No. of Short Trades": 31,
      "Benchmark Return(%)": 1447.07322,
      "Benchmark Return(on $1000)": 14470.732205,
      "Win Rate": 39.344262,
      "Winning Streak": 6,
      "Losing Streak": 5,
      "Gross Profit": 13730.581867,
      "Net Profit": 13385.581867,
      "Average Profit": 219.435768,
      "Maximum Drawdown(%)": 35.034851,
      "Average Drawdown(%)": 6.756261,
      "Largest Win": 6364.174491,
      "Average Win": 1002.866602,
      "Largest Loss": -817.725997,
      "Average Loss": -288.735583,
      "Maximum Holding Time": "94 days 0:0:0",
      "Average Holding Time": "21 days 5:14:45",
      "Maximum Adverse Excursion": 48.841067

In [14]:
#ETHUSDT_12h.csv ETHUSDT_15m.csv ETHUSDT_1d.csv  ETHUSDT_1h.csv  ETHUSDT_2h.csv  ETHUSDT_30m.csv ETHUSDT_4h.csv  ETHUSDT_6h.csv  ETHUSDT_8h.csv

data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/ETH/ETHUSDT_1d.csv")


# Process data
res = process_data(data)

# Apply filtering strategy to remove duplicate signals
res = strat(res)

# Save processed data to CSV for backtesting
res.to_csv("processed_data.csv", index=False)

# Perform backtesting
csv_file_path = "processed_data.csv"
backtest_result = perform_backtest(csv_file_path)

# Print backtest results
for value in backtest_result:
    print(value)


data: {
  "jupyter_id": "team64-5fzelta-5fhpps",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-12-01 00:00:00",
      "Total Trades": 6,
      "Leverage Applied": 5.0,
      "Winning Trades": 4,
      "Losing Trades": 2,
      "No. of Long Trades": 3,
      "No. of Short Trades": 3,
      "Benchmark Return(%)": 1462.97378,
      "Benchmark Return(on $1000)": 14629.737803,
      "Win Rate": 66.666667,
      "Winning Streak": 3,
      "Losing Streak": 1,
      "Gross Profit": 36778.225843,
      "Net Profit": 36751.975843,
      "Average Profit": 6125.329307,
      "Maximum Drawdown(%)": 1.817556,
      "Average Drawdown(%)": 0.609069,
      "Largest Win": 36219.928619,
      "Average Win": 9422.600501,
      "Largest Loss": -676.493231,
      "Average Loss": -469.213081,
      "Maximum Holding Time": "377 days 0:0:0",
      "Average Holding Time": "197 days 20:0:0",
      "Maximum Adverse Excursion": 27.767227,
 